In [1]:
from google.colab import files; files.upload()  # choose your liar.zip


Saving archive.zip to archive.zip


{'archive.zip': b'PK\x03\x04-\x00\x00\x00\x08\x00Hz\xd1R\xb1\xa4\xbd\xcc\xff\xff\xff\xff\xff\xff\xff\xff\x06\x00\x14\x00README\x01\x00\x10\x00\x86\x06\x00\x00\x00\x00\x00\x00D\x03\x00\x00\x00\x00\x00\x00\xadU\xdbn\xdb8\x10}\xe7W\x0c\xf2\xd2\x14P$;\x97^\x0c\x04\xbb\x8a/h\xda\xd4-\x12\xa3A\xb1\xd8\x07\x9a\xa6,&\x14\xa9%\x87\xd6\xfa\xefw(\xc9\x8a\xd3}\xad\x01\x0b\x16gxf\xe6\x9cC\xfa\xee6\xbf\x9f@\x0e7\xf3\xe5\xf4\xd3\xd7\xfc\xfe\x0b\xcc\xf2U\xfe0_\xc1\xe2\xdb=,\xf2/sX\xce\x1f\x1f`6_\xcd\xa7\xab\xdboK\xc6\x1e\x95\xd6\x8aW\xf0\x93\x9b-<\xd2#\x81\x93;\xc5]\x02\xf1\t\xdf\xb9A\x0f\xd6\xc0B9y\x12\xb1\x97\xb2\x81\x1biDYq\xf7\x0c3\x8e\xdcK\x84\xc2:X\xf0g\x19\xc3\x1ef\x12\xa5@eM\x02h\x81\xd7\xb5$(e\xe0\xbb\xb3B\xca\x8d2[\xc2,\x00K\tWWXBnL\xe0\x1a\xbeJ\x89\x14;\x84r\xef\xadP<\xe2\xb4\xf8S[\xd5\x01\xdbwJ\xbe\xa3\xcc\xa0<*\xe1\xe14\x9f\xde\xc1\xf9h\xfc\xfem\x02\xbe\xb4\x0e\xa1\xe6\xb5\xa4\x19~p#l\xd8\xc5\x9f7\xd3\x04\xa6\xdc\xf0\rO\xe0s\xd0{\xb8\x18\x9d\xe5a\x1b<\xc2e\x02\x04\x90\xb2\xeb\xdf\xf1a3\xe

In [2]:
!unzip archive.zip -d liar_data
!ls -lah liar_data


Archive:  archive.zip
  inflating: liar_data/README        
  inflating: liar_data/test.tsv      
  inflating: liar_data/train.tsv     
  inflating: liar_data/valid.tsv     
total 2.9M
drwxr-xr-x 2 root root 4.0K Aug 30 09:21 .
drwxr-xr-x 1 root root 4.0K Aug 30 09:21 ..
-rw-r--r-- 1 root root 1.7K Jun 17  2021 README
-rw-r--r-- 1 root root 295K Jun 17  2021 test.tsv
-rw-r--r-- 1 root root 2.3M Jun 17  2021 train.tsv
-rw-r--r-- 1 root root 295K Jun 17  2021 valid.tsv


In [3]:
import pandas as pd

train = pd.read_csv('liar_data/train.tsv', sep='\t', header=None)
train.columns = [
    'statement_id','label','statement','subject','speaker',
    'speaker_job','state','party','barely_true','false',
    'half_true','mostly_true','pants_on_fire','context'
]

train.head()


,statement_id,label,statement,subject,speaker,speaker_job,state,party,barely_true,false,half_true,mostly_true,pants_on_fire,context
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN


In [4]:
train['label'].value_counts()


,count
label,
half-true,2114
false,1995
mostly-true,1962
true,1676
barely-true,1654
pants-fire,839


In [5]:
label_map = {
    'true': 'real',
    'mostly-true': 'real',
    'half-true': 'real',
    'false': 'fake',
    'barely-true': 'fake',
    'pants-on-fire': 'fake'
}

train['label'] = train['label'].map(label_map)
train = train.dropna(subset=['label'])  # remove any unmapped values

train['label'].value_counts()


,count
label,
real,5752
fake,3649


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

X = train['statement']
y = train['label']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=42
)

tfidf = TfidfVectorizer(stop_words='english', max_features=5000)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)


In [7]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)
model.fit(X_train_tfidf, y_train)


LogisticRegression(max_iter=1000)

In [8]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = model.predict(X_test_tfidf)

print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Classification Report:
               precision    recall  f1-score   support

        fake       0.52      0.26      0.35       730
        real       0.64      0.85      0.73      1151

    accuracy                           0.62      1881
   macro avg       0.58      0.55      0.54      1881
weighted avg       0.60      0.62      0.58      1881

Confusion Matrix:
 [[191 539]
 [176 975]]


In [9]:
def predict_news(text):
    vec = tfidf.transform([text])
    pred = model.predict(vec)[0]
    return f"Predicted Label: {pred}"

print(predict_news("The government is giving free laptops to all citizens."))
print(predict_news("WHO launches a new vaccination program for 2025."))


Predicted Label: fake
Predicted Label: real


In [10]:
!pip install gradio


In [11]:
import gradio as gr

def fake_news_checker(text):
    vec = tfidf.transform([text])
    pred = model.predict(vec)[0]
    return f"This news is likely: {pred.upper()}"

# Create interface
demo = gr.Interface(
    fn=fake_news_checker,
    inputs=gr.Textbox(lines=3, placeholder="Paste news headline or statement here..."),
    outputs="text",
    title="📰 Fake News Detection (TF-IDF + Logistic Regression)",
    description="Enter a news headline or short article statement to check if it's real or fake."
)

demo.launch(share=True)  # share=True gives you a public link


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://49f98c5fe51d131ed7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
